# 利用CNN实现MNIST数据集的手写数字识别任务

In [23]:
import numpy as np
import torch
from torch import nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

## 读取数据
-分别构建训练集和测试集
-DataLoader来迭代取数据

In [24]:
# 定义超参数
input_size = 28
num_classes = 10
num_epochs = 3
batch_size = 64

# 训练集
train_dataset = datasets.MNIST(root='./data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)
# 测试集
test_dataset = datasets.MNIST(root='./data',
                              train=False,
                              transform=transforms.ToTensor(),
                              download=True)

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=True)

In [25]:
train_dataset[2][0].shape # 第二个样本的数据

torch.Size([1, 28, 28])

In [26]:
test_dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()

## 卷积网络模块的构建
-一般卷积层，relu层，池化层可以写成一个套餐
-注意卷积最后结果还是一个特征图，需要把图转换为向量才能做分类或者回归任务

In [27]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential( # 输入的大小为(1,28,28)
            nn.Conv2d(in_channels=1,out_channels=16,kernel_size=5,stride=1,padding=2), # 输出的维度为(16,28,28)
            nn.ReLU(), # 输出的维度(16,28,28)
            nn.MaxPool2d(kernel_size=2) # 输出的维度(16,14,14)
        )
        self.conv2 = nn.Sequential( # 输入的维度(16,14,14)
            nn.Conv2d(in_channels=16,out_channels=32,kernel_size=5,stride=1,padding=2), # 输出的维度(32,14,14)
            nn.ReLU(), # 输出的维度(32,14,14)
            nn.MaxPool2d(kernel_size=2) # 输出的维度(32,7,7)
        )
        self.out = nn.Linear(32*7*7,10) # 全连接
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0),-1) # flatten操作，结果为：(batch_size,32*7*7)
        output = self.out(x)
        return output

## 准确率函数

In [28]:
def accuracy(predictions,labels):
    pred = torch.max(predictions.data,1)[1] # 得到最大值的下标
    rights = pred.eq(labels.data.view_as(pred)).sum() # 得到预测准确的数目
    return rights,len(labels)

## 训练和验证函数

In [37]:
def train_val(train_loader,test_loader,criterion,optimizer):
    # 当前的epoch的结果保存下来
    train_rights = []
    
    for batch_idx,(data,target) in tqdm(enumerate(train_loader),total = len(train_loader), position = 0, leave = True):
        net.train()
        output = net(data)
        loss = criterion(output,target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        right= accuracy(output,target)
        train_rights.append(right)
    
        if batch_idx % 100 == 0:
            # 验证/测试
            val_rights = []
            for (data,target) in test_loader:
                output = net(data)
                right = accuracy(output,target)
                val_rights.append(right)
                
            # 准确率计算
            # 获取训练集的正确个数和总样本数，得到一个元组
            train_r = (sum([tup[0]for tup in train_rights]),sum([tup[1] for tup in train_rights]))
            # 获得验证集的正确个数和总样本数，得到一个元组
            val_r = (sum([tup[0]for tup in val_rights]),sum([tup[1] for tup in val_rights]))   
            
            print(f'当前epoch:{epoch},损失：{loss.data.item()},训练集准确率：{np.round(100.*train_r[0].numpy()/train_r[1],5)}%,测试集准确率{np.round(100.*val_r[0].numpy()/val_r[1],5)}%')

## 训练网络模型

In [38]:
# 实例化
net = CNN()
# 损失函数
criterion = nn.CrossEntropyLoss()
# 优化器
optimizer = optim.Adam(net.parameters(),lr=0.005)

# 开始训练
for epoch in range(1,num_epochs+1):
    train_val(train_loader,test_loader,criterion,optimizer)

  1%|▌                                                                                 | 7/938 [00:01<02:55,  5.32it/s]

当前epoch:1,损失：2.2879769802093506,训练集准确率：14.0625%,测试集准确率11.96%


 12%|█████████▎                                                                      | 109/938 [00:05<01:02, 13.17it/s]

当前epoch:1,损失：0.16246330738067627,训练集准确率：84.90099%,测试集准确率95.6%


 22%|█████████████████▉                                                              | 210/938 [00:08<01:00, 12.09it/s]

...

 86%|████████████████████████████████████████████████████████████████████▋           | 806/938 [00:34<00:13,  9.51it/s]

当前epoch:3,损失：0.02716967463493347,训练集准确率：98.90566%,测试集准确率98.83%


 97%|█████████████████████████████████████████████████████████████████████████████▎  | 906/938 [00:38<00:03, 10.35it/s]

当前epoch:3,损失：0.050107236951589584,训练集准确率：98.89012%,测试集准确率98.88%


100%|████████████████████████████████████████████████████████████████████████████████| 938/938 [00:39<00:00, 23.97it/s]
